In [1]:
import pandas as pd
import numpy as np
#add other dependencies for visuals

In [2]:
#path_link="Data/Seasons_Episodes/output/charlines_df.csv"
#df_charlines = pd.read_csv(path_link)
df_charlines = pd.read_csv("https://raw.githubusercontent.com/limesarelife/SIADS_696_Milestone2/main/Data/Seasons_Episodes/output/charlines_df.csv")

In [3]:
#df_charlines.info()

In [4]:
df_charlines.head(3)

,Unnamed: 0,index,Character,Dialog,Season,Episode,Fix Name,Character_Fix,Spoke_Before,Character_Keep,Length Dialog
0,0,3,HOPPER,NaN,3,5,True,HOPPER,NaN,Keep,0
1,0,3,HOPPER,NaN,3,5,True,HOPPER,NaN,Keep,0
2,0,3,HOPPER,NaN,3,5,True,HOPPER,NaN,Keep,0


In [5]:
df_charlines.loc[df_charlines["Spoke_Before"].isnull(),'Spoke_Before'] = df_charlines['Character_Fix']

In [6]:
df_charlines.head(3)

,Unnamed: 0,index,Character,Dialog,Season,Episode,Fix Name,Character_Fix,Spoke_Before,Character_Keep,Length Dialog
0,0,3,HOPPER,NaN,3,5,True,HOPPER,HOPPER,Keep,0
1,0,3,HOPPER,NaN,3,5,True,HOPPER,HOPPER,Keep,0
2,0,3,HOPPER,NaN,3,5,True,HOPPER,HOPPER,Keep,0


In [7]:
df_charlines.tail(3)

,Unnamed: 0,index,Character,Dialog,Season,Episode,Fix Name,Character_Fix,Spoke_Before,Character_Keep,Length Dialog
6065,39,270,CLAUDIA,Where did you see it last?,2,9,True,CLAUDIA,SUSAN,Keep,26
6066,39,270,CLAUDIA,What's in there that's so important anyway? ...,2,9,True,CLAUDIA,SUSAN,Keep,132
6067,39,270,CLAUDIA,That's your ride! Dusty??? The bag from Melva...,2,9,True,CLAUDIA,SUSAN,Keep,75


In [8]:
df_charlines.dropna(subset=['Dialog'],inplace = True)

In [9]:
df_charlines.drop(['Unnamed: 0'], axis=1, inplace = True)
df_charlines.shape

(5457, 10)

In [10]:
df_charlines['Dialog_Check'] = df_charlines['Dialog'].apply(lambda x: x.isdigit())

In [11]:
df_charlines['Remove_Lines'] = df_charlines['Dialog'].apply(lambda x: True if "1/" in x else False)

In [12]:
#df_charlines[df_charlines['Remove_Lines']==True]
#removing odd script lines when based on location setting that were caught
df_charlines2 = df_charlines[df_charlines['Remove_Lines']==False].copy()

In [13]:
# reordering columns
cols = list(df_charlines2)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Character_Fix')))
#print(cols)
#['Character_Fix', 'Character', 'Dialog', 'Season', 'Episode', 'Fix Name', 'Length Dialog', 'Character_Keep', 
#'Dialog_Check', 'Remove_Lines']
# use index to reorder
df_charlines2 = df_charlines2.loc[:, cols]

In [14]:
df_charlines2.head(3)

,Character_Fix,index,Character,Dialog,Season,Episode,Fix Name,Spoke_Before,Character_Keep,Length Dialog,Dialog_Check,Remove_Lines
5,HOPPER,3,HOPPER,Looks like somebody’s home. As the truck parks...,3,5,True,HOPPER,Keep,341,False,False
6,HOPPER,3,HOPPER,Where’s that coming from? Joyce pauses at the ...,3,5,True,HOPPER,Keep,165,False,False
7,HOPPER,3,HOPPER,"Hey, dipshits! The men drop their tools and st...",3,5,True,HOPPER,Keep,80,False,False


In [15]:
df_charlines2.shape

(5451, 12)

In [17]:
import plotly as py
import plotly.express as px
import numpy as np
import re
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from collections import Counter
from nltk import word_tokenize
import nltk

#order = df["Character_Fix"].unique()
#dialog = df["Dialog"]
#dialogCounts = dialog.value_counts()
#dialogCounts = dialogCounts.sort_values(ascending = False)[0:50]

In [18]:
df = df_charlines2.copy()
df['Dialog_clean'] = df['Dialog'].apply(lambda x: re.sub(r'[^\w\s]','',x))
df['Dialog_tokenized'] = df['Dialog_clean'].apply(lambda x: word_tokenize(x.strip().lower()))
df['Dialog_Count'] = df['Dialog_tokenized'].apply(lambda x: len(x))
LinesSpokeDf = df.groupby(['Character_Fix'])["Dialog_Count"].sum().reset_index()
LinesSpokeDf = LinesSpokeDf.sort_values(by=["Dialog_Count"], ascending = False)

In [19]:
_20df = LinesSpokeDf.head(20)

In [20]:
top20chars = list(_20df.Character_Fix.unique())

In [21]:
print(top20chars)

['HOPPER', 'MIKE', 'STEVE', 'DUSTIN', 'JOYCE', 'NANCY', 'ROBIN', 'MAX', 'JONATHAN', 'LUCAS', 'WILL', 'ELEVEN', 'MURRAY', 'BILLY', 'ERICA', 'KAREN', 'DR BRENNER', 'MAYOR KLINE', 'DR OWENS', 'TOM']


In [22]:
top20df = df[df['Character_Fix'].isin(top20chars)].copy()

In [23]:
print(list(top20df.Character_Fix.unique()))

['HOPPER', 'JOYCE', 'JONATHAN', 'ROBIN', 'STEVE', 'NANCY', 'TOM', 'MAX', 'ELEVEN', 'DUSTIN', 'ERICA', 'LUCAS', 'WILL', 'KAREN', 'MIKE', 'MURRAY', 'BILLY', 'MAYOR KLINE', 'DR BRENNER', 'DR OWENS']


In [24]:
top20dfv2 = top20df[top20df['Season']==3]

In [25]:
### DR BRENNER is assumed to be dead in season 3 so it makes sense he is not in the top 20
### characters for season 3 and then dr owens is only is season 2 and season 4 and we 
### briefly see him in season 3 but not speaking.

drb = top20df[top20df['Character_Fix']=="DR BRENNER"]
dro = top20df[top20df['Character_Fix']=="DR OWENS"]
print(drb.Season.unique())
print(dro.Season.unique())

[1 4]
[4 2]


In [26]:
print(list(top20dfv2.Character_Fix.unique()))

['HOPPER', 'JOYCE', 'JONATHAN', 'ROBIN', 'STEVE', 'NANCY', 'TOM', 'MAX', 'ELEVEN', 'DUSTIN', 'ERICA', 'LUCAS', 'WILL', 'KAREN', 'MIKE', 'MURRAY', 'BILLY', 'MAYOR KLINE']


In [27]:
top20df_S3 = top20dfv2.groupby(['Character_Fix','Episode'])['Dialog_Count'].sum().reset_index()

In [28]:
top20df_S3.head()

,Character_Fix,Episode,Dialog_Count
0,BILLY,1,447
1,BILLY,2,246
2,BILLY,3,236
3,BILLY,4,340
4,BILLY,6,447


In [29]:
gender = pd.read_csv('https://raw.githubusercontent.com/limesarelife/SIADS_696_Milestone2/main/Data/Seasons_Episodes/output/gender_st.csv')

In [30]:
gender

,Character_Fix,Gender,Age
0,AGENT FRAZIER,Female,1
1,AGENT HARMON,Male,1
2,ALEXEI,Male,1
3,ANDY,Male,2
4,ARGYLE,Male,2
...,...,...,...
57,VICKIE,Female,2
58,VICTOR CREEL,Male,1
59,WAYNE,Male,1
60,WILL,Male,2


In [31]:
gender_20df = pd.merge(_20df, gender, on="Character_Fix")

In [32]:
gender_20df['Gender'] = gender_20df['Gender'].apply(lambda x: x.strip())

In [33]:
gender

,Character_Fix,Gender,Age
0,AGENT FRAZIER,Female,1
1,AGENT HARMON,Male,1
2,ALEXEI,Male,1
3,ANDY,Male,2
4,ARGYLE,Male,2
...,...,...,...
57,VICKIE,Female,2
58,VICTOR CREEL,Male,1
59,WAYNE,Male,1
60,WILL,Male,2


In [34]:
fulldf_gen = pd.merge(df, gender, on="Character_Fix")
fulldf_gen['Gender'] = fulldf_gen['Gender'].apply(lambda x: x.strip())
eggos = fulldf_gen.groupby(['Age'])['Dialog_Count'].sum().reset_index()

In [35]:
Total = eggos['Dialog_Count'].sum()
eggos['percent'] = eggos.Dialog_Count.apply(lambda x: x/Total)


In [36]:
sent_dict = top20dfv2.groupby('Character_Fix')['Dialog_clean'].apply(list).reset_index()#.apply(list).to_dict()

In [37]:
sent_dict['Dialog_clean'] = sent_dict['Dialog_clean'].apply(lambda x: ".".join(str(e) for e in x))

In [38]:
sent_dict.head()

,Character_Fix,Dialog_clean
0,BILLY,Dont be afraid Itll be over soon Just try and...
1,DUSTIN,Shit Shit.Why dont these buttons work.What do ...
2,ELEVEN,JONATHAN . KAREN WHEELER .I found him MIKE WI...
3,ERICA,Press the button.JUST THEN the elevator comes ...
4,HOPPER,Looks like somebodys home As the truck parks w...


In [39]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()


def polarity(text):
    pol = sid.polarity_scores(text)
    return pol
sent_dict['Dialog_clean'] = sent_dict['Dialog_clean'].apply(lambda x: x.lower())
sent_dict['polarity'] = sent_dict['Dialog_clean'].apply(polarity)  #polarity checking
sent_dict['compound']  = sent_dict['polarity'].apply(lambda score_dict: score_dict['compound'])

sent_dict['sentiment'] = sent_dict['compound'].apply(lambda x: "Positive" if x>=0.5 \
                                                     else("Negative" if x<=-0.5 \
                                                     else "Neutral"))
print("polarity checked successfull")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jacquelineskunda/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


polarity checked successfull


In [40]:
sent_dict.head(3)

,Character_Fix,Dialog_clean,polarity,compound,sentiment
0,BILLY,dont be afraid itll be over soon just try and...,"{'neg': 0.116, 'neu': 0.815, 'pos': 0.069, 'co...",-0.9982,Negative
1,DUSTIN,shit shit.why dont these buttons work.what do ...,"{'neg': 0.091, 'neu': 0.803, 'pos': 0.106, 'co...",0.9970,Positive
2,ELEVEN,jonathan . karen wheeler .i found him mike wi...,"{'neg': 0.088, 'neu': 0.819, 'pos': 0.093, 'co...",0.9724,Positive


### Start of supervised Learning

In [41]:
df_show = df[['Character_Fix','Season',"Episode","Dialog"]].copy()

In [42]:
df_show.head(3)

,Character_Fix,Season,Episode,Dialog
5,HOPPER,3,5,Looks like somebody’s home. As the truck parks...
6,HOPPER,3,5,Where’s that coming from? Joyce pauses at the ...
7,HOPPER,3,5,"Hey, dipshits! The men drop their tools and st..."


In [43]:
# Function to preprocess text
def cleaner(text):
  	# Create doc object
    doc = nlp(text)
    
    # Generate lemmas and lower case and remove punctuation
    lemmas = [token.lemma_.lower() for token in doc if not token.is_punct\
              and token.is_alpha]
    #token.is_stop == False 
    # Remove stopwords, extra characters like numbers (second check)
    #all_lemmas = [lemma for lemma in lemmas if lemma.isalpha]
    
    return " ".join(lemmas)

In [44]:
import spacy
from spacy import displacy
import seaborn as sns
import string
import nltk
import en_core_web_sm
#nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm')

In [45]:
df_show['token_lemma'] = df_show['Dialog'].apply(cleaner)

In [46]:
df_show.head(3)

,Character_Fix,Season,Episode,Dialog,token_lemma
5,HOPPER,3,5,Looks like somebody’s home. As the truck parks...,look like somebody home as the truck park we r...
6,HOPPER,3,5,Where’s that coming from? Joyce pauses at the ...,where that come from joyce pause at the foot o...
7,HOPPER,3,5,"Hey, dipshits! The men drop their tools and st...",hey dipshit the man drop their tool and stand ...


Supervised learning classification techniques:   
Logistic Regression: to understand the strength of relationships by leveraging signals to approximate the most appropriate classification. This will be our baseline for classification.    
Random Forest Classifier: to perform classification tasks on noisy and/or highly dimensional data by making use of an ensemble of decision trees for classification.     
LinearSVM based regression: generalization capability of the words/lines with high prediction accuracy for character name, robust to outliers. Will need to transform all lines of text into vectors in order to encode and leverage the power of SVM.     

Evaluation Metrics:
F1 Score : mean of precision and recall scores    
Precision : explains how many of the correctly predicted cases actually turned out to be positive. Precision is useful where False Positives are a higher concern than False Negatives.     
AUC (Area Under the Curve): to measure how well the classifier was able to distinguish different classes, if our dataset proves to be imbalanced which might be the case since characters do speak different volumes of lines etc.    
K-Fold cross validation accuracy: to measure and test if our various models' accuracy would be better, worse or the same if we had used a different section of the data set as a validation set. Evaluation is performed using different shuffling and chunking of the dataset through various iterations. This will be great because we can see the various scoring/evaluation metrics above at once (such as accuracy, precision, recall and f1)    
Visualizations:
Confusion matrix: in order to compare actual label versus predicted label (TP, TN, FP, FN) and identify proper (best) evaluation metrics/scoring to use.  
Histograms: to show frequency of characters predicted (by probability with use of bins)
We will try to inspect model performance via visualizations between different characters (classes).   
Line charts for visualizing AUC-ROC and other results/metrics.    
All visualizations will keep the Stranger Things aesthetic as well. Will look into building a visualization which is composed of characters face/faces with model output probabilities showing the confidence prediction. Or sentiment of characters lines spoken, we are sure there are characters which have more negative sentiment overall than others. We will scope out over the course of project work.    

### Random Forest Classifier 

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import model_selection
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [48]:
RANDOM_SEED = 655

In [49]:
df_show.head(3)

,Character_Fix,Season,Episode,Dialog,token_lemma
5,HOPPER,3,5,Looks like somebody’s home. As the truck parks...,look like somebody home as the truck park we r...
6,HOPPER,3,5,Where’s that coming from? Joyce pauses at the ...,where that come from joyce pause at the foot o...
7,HOPPER,3,5,"Hey, dipshits! The men drop their tools and st...",hey dipshit the man drop their tool and stand ...


In [50]:
df_show['length'] = df_show.token_lemma.apply(lambda x: len(x))

In [51]:
check = df_show.groupby(['length']).min().reset_index()

In [52]:
check.iloc[0:5,:]

,length,Character_Fix,Season,Episode,Dialog,token_lemma
0,0,STEVE,3,5,2/8,
1,2,BILLY,1,1,-- Hi...,go
2,3,DEPUTY CALLAHAN,1,1,... God --,and
3,4,ARGYLE,1,1,Boop.,boop
4,5,BILLY,1,1,", LUCAS",a man


In [53]:
df_show2= df_show.explode(column='token_lemma').dropna(subset=['token_lemma'])

In [54]:
df_show2.head(3)

,Character_Fix,Season,Episode,Dialog,token_lemma,length
5,HOPPER,3,5,Looks like somebody’s home. As the truck parks...,look like somebody home as the truck park we r...,303
6,HOPPER,3,5,Where’s that coming from? Joyce pauses at the ...,where that come from joyce pause at the foot o...,150
7,HOPPER,3,5,"Hey, dipshits! The men drop their tools and st...",hey dipshit the man drop their tool and stand ...,72


In [55]:
train_df, test_df = \
              np.split(df_show.sample(frac=1, random_state=RANDOM_SEED), 
                       [int(.8*len(df_show))])
print(len(train_df), len(test_df))

4360 1091


In [57]:
vectorizer = TfidfVectorizer(min_df = 20, stop_words = 'english')
X_train = vectorizer.fit_transform(train_df.token_lemma)
X_test = vectorizer.transform(test_df.token_lemma)

y_train=train_df['Character_Fix'].values
y_test=test_df['Character_Fix'].values


In [60]:
# Instantiate Random forest classifier 
clf=RandomForestClassifier(n_estimators=100, random_state = RANDOM_SEED)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [61]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.1998166819431714


### Terrible score now we hyperparameter tune!

In [63]:
param_grid_ = {'n_estimators': [500, 800, 1500, 2500, 5000],
               'max_features': ['auto', 'sqrt', 'log2'],
               'max_depth':[10, 20, 30, 40, 50],
               'min_samples_split': [2, 5, 10, 15, 20],
               'min_samples_leaf': [1, 2, 5, 10, 15],
               'criterion':["gini", "entropy", "log_loss"]
              }

In [ ]:
RCF = RandomForestClassifier(random_state=1)
RCF_random = RandomizedSearchCV(estimator=RCF,
                                param_distributions=param_grid_,
                               n_iter=100,
                               cv=5,
                               verbose=2,
                               random_state=RANDOM_SEED,
                               )

RCF_random.fit(X_train,y_train)

print(RCF_random.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



[CV] END criterion=entropy, max_depth=30, max_features=sqrt, min_samples_leaf=15, min_samples_split=15, n_estimators=1500; total time=   6.6s
[CV] END criterion=entropy, max_depth=30, max_features=sqrt, min_samples_leaf=15, min_samples_split=15, n_estimators=1500; total time=   9.6s
[CV] END criterion=entropy, max_depth=30, max_features=sqrt, min_samples_leaf=15, min_samples_split=15, n_estimators=1500; total time=   7.4s
[CV] END criterion=entropy, max_depth=30, max_features=sqrt, min_samples_leaf=15, min_samples_split=15, n_estimators=1500; total time=   9.5s
[CV] END criterion=entropy, max_depth=30, max_features=sqrt, min_samples_leaf=15, min_samples_split=15, n_estimators=1500; total time=   9.4s
[CV] END criterion=entropy, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1500; total time=  20.7s
[CV] END criterion=entropy, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1500; total time=  21.4s
[CV] END

[CV] END criterion=log_loss, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=2500; total time=   0.9s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=20, n_estimators=800; total time=   4.8s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=20, n_estimators=800; total time=   4.9s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=20, n_estimators=800; total time=   6.1s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=20, n_estimators=800; total time=   6.3s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=20, n_estimators=800; total time=   5.4s
[CV] END criterion=gini, max_depth=40, max_features=auto, min_samples_leaf=15, min_samples_split=15, n_estimators=1500; total time=   9.2s
[CV] END criterion=

[CV] END criterion=log_loss, max_depth=50, max_features=auto, min_samples_leaf=15, min_samples_split=10, n_estimators=800; total time=   0.3s
[CV] END criterion=log_loss, max_depth=50, max_features=auto, min_samples_leaf=15, min_samples_split=10, n_estimators=800; total time=   0.3s
[CV] END criterion=gini, max_depth=40, max_features=auto, min_samples_leaf=10, min_samples_split=20, n_estimators=1500; total time=   6.3s
[CV] END criterion=gini, max_depth=40, max_features=auto, min_samples_leaf=10, min_samples_split=20, n_estimators=1500; total time=   6.1s
[CV] END criterion=gini, max_depth=40, max_features=auto, min_samples_leaf=10, min_samples_split=20, n_estimators=1500; total time=   5.9s
[CV] END criterion=gini, max_depth=40, max_features=auto, min_samples_leaf=10, min_samples_split=20, n_estimators=1500; total time=   5.6s
[CV] END criterion=gini, max_depth=40, max_features=auto, min_samples_leaf=10, min_samples_split=20, n_estimators=1500; total time=   6.6s
[CV] END criterion=gi

[CV] END criterion=log_loss, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2, n_estimators=500; total time=   0.2s
[CV] END criterion=log_loss, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2, n_estimators=500; total time=   0.2s
[CV] END criterion=entropy, max_depth=20, max_features=auto, min_samples_leaf=15, min_samples_split=10, n_estimators=1500; total time=   7.1s
[CV] END criterion=entropy, max_depth=20, max_features=auto, min_samples_leaf=15, min_samples_split=10, n_estimators=1500; total time=   6.8s
[CV] END criterion=entropy, max_depth=20, max_features=auto, min_samples_leaf=15, min_samples_split=10, n_estimators=1500; total time=   7.4s
[CV] END criterion=entropy, max_depth=20, max_features=auto, min_samples_leaf=15, min_samples_split=10, n_estimators=1500; total time=   7.7s
[CV] END criterion=entropy, max_depth=20, max_features=auto, min_samples_leaf=15, min_samples_split=10, n_estimators=1500; total time=   7.3s
[CV] END c

[CV] END criterion=log_loss, max_depth=30, max_features=sqrt, min_samples_leaf=10, min_samples_split=5, n_estimators=800; total time=   0.2s
[CV] END criterion=log_loss, max_depth=30, max_features=sqrt, min_samples_leaf=10, min_samples_split=5, n_estimators=800; total time=   0.3s
[CV] END criterion=log_loss, max_depth=30, max_features=sqrt, min_samples_leaf=10, min_samples_split=5, n_estimators=800; total time=   0.2s
[CV] END criterion=gini, max_depth=20, max_features=log2, min_samples_leaf=10, min_samples_split=2, n_estimators=800; total time=   2.0s
[CV] END criterion=gini, max_depth=20, max_features=log2, min_samples_leaf=10, min_samples_split=2, n_estimators=800; total time=   2.0s
[CV] END criterion=gini, max_depth=20, max_features=log2, min_samples_leaf=10, min_samples_split=2, n_estimators=800; total time=   2.0s
[CV] END criterion=gini, max_depth=20, max_features=log2, min_samples_leaf=10, min_samples_split=2, n_estimators=800; total time=   2.0s
[CV] END criterion=gini, max_

[CV] END criterion=log_loss, max_depth=40, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=1500; total time=   0.6s
[CV] END criterion=log_loss, max_depth=40, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=1500; total time=   0.6s
[CV] END criterion=log_loss, max_depth=40, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=1500; total time=   0.5s
[CV] END criterion=log_loss, max_depth=40, max_features=log2, min_samples_leaf=5, min_samples_split=10, n_estimators=1500; total time=   0.6s
[CV] END criterion=entropy, max_depth=50, max_features=auto, min_samples_leaf=15, min_samples_split=20, n_estimators=800; total time=   4.5s
[CV] END criterion=entropy, max_depth=50, max_features=auto, min_samples_leaf=15, min_samples_split=20, n_estimators=800; total time=   4.3s
[CV] END criterion=entropy, max_depth=50, max_features=auto, min_samples_leaf=15, min_samples_split=20, n_estimators=800; total time=   4.5s
[CV] END 

[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=5000; total time=  29.0s
[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=5000; total time=  30.3s
[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=5000; total time=  24.0s
[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=5000; total time=  27.0s
[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=5000; total time=  28.3s
[CV] END criterion=gini, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=20, n_estimators=1500; total time=   4.3s
[CV] END criterion=gini, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=20, n_estimators=1500; total time=   4.2s


### Accuracy is still terrible so we will do two things to try and help out the data set.   
### 1. Only use characters who are top 20 characters (1st attempt)
### 2. 1st attempt along with only getting spoken dialog where there are more than 2 or 3 words.

### We can also add the gender along with season and episode and age (1 = adult, 2 = child/teenage, 0 = no age)

In [65]:
df = pd.DataFrame(list(zip(y_test,y_pred)), columns = ['Character', 'Predicted'])
#df['Error'] = df['Character'] - df['Predicted']

#Plot the joint distribution
#Let's start by seeing how the two distributions line up
#sns.barplot(data=df, x='Character', y='Predicted', kind="hex", color="red",
#             )

In [66]:
df_tf = pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out())

In [67]:
df_tf.shape

(4360, 285)

In [68]:
df_tf.head(3)

,actually,ago,ahead,alexei,answer,approach,arm,ask,attention,away,...,whoa,window,word,work,world,wrong,yeah,year,yes,yuri
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.717594,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [69]:
train_df.shape

(4360, 6)

In [70]:
tr_df = train_df.reset_index()

In [71]:
train_w_feat = pd.concat([df_tf,tr_df],axis=1)

In [72]:
train_w_feat.shape

(4360, 292)

In [73]:
train_w_feat.drop(['Dialog','token_lemma','index'], axis=1, inplace=True)

In [74]:
train_w_features = pd.merge(train_w_feat, gender, on="Character_Fix",how='left')

In [75]:
fix = train_w_features[train_w_features.Gender.isnull()]

In [76]:
fix

,actually,ago,ahead,alexei,answer,approach,arm,ask,attention,away,...,yeah,year,yes,yuri,Character_Fix,Season,Episode,length,Gender,Age


In [77]:
#train_w_features2 =train_w_features[train_w_features['length']>=3].copy()

In [78]:
train_w_features3 = train_w_features.drop(['Character_Fix'], axis=1).copy()

In [79]:
train_w_features3['Gender'].replace(['Female ', 'Female','Male','Male ','Neutral'],
                        [1, 1,2,2,0], inplace=True)

In [80]:
train_w_features3.head(3)

,actually,ago,ahead,alexei,answer,approach,arm,ask,attention,away,...,wrong,yeah,year,yes,yuri,Season,Episode,length,Gender,Age
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.717594,0.0,1,2,36,1,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,4,4,19,2,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,3,3,159,2,2


In [81]:
RCF_model_v2 = RandomForestClassifier(n_estimators=100, \
                              random_state=RANDOM_SEED,\
                              max_features='sqrt').fit(train_w_features3, y_train)

In [82]:
len(y_train)

4360

In [83]:
len(y_test)

1091

In [89]:
df_tf_test = pd.DataFrame(X_test.toarray(), columns=vectorizer.get_feature_names_out())

In [90]:
tr_df_test = test_df.reset_index()

In [91]:
test_w_feat = pd.concat([df_tf_test,tr_df_test],axis=1)
test_w_feat.drop(['Dialog','token_lemma','index'], axis=1, inplace=True)
test_w_features = pd.merge(test_w_feat, gender, on="Character_Fix",how='left')
test_w_features3 = test_w_features.drop(['Character_Fix'], axis=1).copy()
test_w_features3['Gender'].replace(['Female ', 'Female','Male','Male ','Neutral'],
                        [1, 1,2,2,0], inplace=True)
test_w_features3.shape

(1091, 290)

In [92]:
y_pred_RCF_modelv2 =RCF_model_v2.predict(test_w_features3)

In [93]:
len(y_pred_RCF_modelv2)

1091

In [94]:
accm2 = metrics.accuracy_score(y_test,y_pred_RCF_modelv2)

#lg_model_scr2 = lg_model.score(test_w_features3, y_test)

In [95]:
accm2

0.47296058661778184

In [ ]:
train_score_no_tune_with_features = RCF_model_v2.score(train_w_features3, y_train)

In [ ]:
train_score_no_tune_with_features

### Try this with our new hyper parameters since it went from 17% to 19% and is now out 39.5% with adding some features to the data.


In [99]:

RCF_model_tuned = RandomForestClassifier(n_estimators=100, \
                              random_state=RANDOM_SEED,\
                              max_features='sqrt').fit(train_w_features3, y_train)

y_pred_RCF_model_tuned = RCF_model_tuned.predict(test_w_features3)


accm_tuned = metrics.accuracy_score(y_test,y_pred_RCF_model_tuned)

print(accm_tuned)

acc_train = RCF_model_tuned.score(train_w_features3, y_train)

print(acc_train)

# lg_model_tuned = LogisticRegression(max_iter = 500, \ class_weight = 'balanced',
#                               random_state=RANDOM_SEED,\
#                               solver = 'liblinear',penalty= 'l1',
#                               C = 0.1, verbose = 15, ).fit(train_w_features3, y_train)

# y_pred_lg_model_tuned = lg_model_tuned.predict(test_w_features3)

# accm_tuned = metrics.accuracy_score(y_test,y_pred_lg_model_tuned)

# print(accm_tuned)

# acc_train = lg_model_tuned.score(train_w_features3, y_train)

# print(acc_train)

0.47296058661778184
0.9855504587155963


### It only performed 3.2% better so we are not getting far, I think dropping data from the org data based on length of line spoken and how often a character occurs along with text difficulty as well as keeping our features we already just used (gender, season, episode) will help this process along.  We can try and add in who spoke durng scene as well to see if it helps.  We would be thrilled if this was able to get over 50%.  We knew after performing EDA and QAing the data there were issues with just using pdf's in general and then through the show there actually isnt much dialog as one would hope, it is more scene setting and action.

### Next drop out rows where length is less than 2 or 3 and redo training/hyperparameter tuning and/or characters who speak less than 5 times, maybe standard scale new df

### I think what also needs to happen is hyper parameter tuning with these features and data

In [100]:
### starting with the data and features - not split yet.
df_show.head()

,Character_Fix,Season,Episode,Dialog,token_lemma,length
5,HOPPER,3,5,Looks like somebody’s home. As the truck parks...,look like somebody home as the truck park -pro...,322
6,HOPPER,3,5,Where’s that coming from? Joyce pauses at the ...,where that come from joyce pause at the foot o...,153
7,HOPPER,3,5,"Hey, dipshits! The men drop their tools and st...",hey dipshit the man drop -pron- tool and stand...,72
8,HOPPER,3,5,"Dr. Alexei steps forward, putting himself betw...",alexei step forward put -pron- between hopper ...,148
10,HOPPER,3,5,English! You speak English? Alexei tries to ca...,english -pron- speak english alexei try to cal...,61


In [101]:
df_sh2 = df_show.merge(gender, how="left", on ='Character_Fix')

In [102]:
df_sh2

,Character_Fix,Season,Episode,Dialog,token_lemma,length,Gender,Age
0,HOPPER,3,5,Looks like somebody’s home. As the truck parks...,look like somebody home as the truck park -pro...,322,Male,1
1,HOPPER,3,5,Where’s that coming from? Joyce pauses at the ...,where that come from joyce pause at the foot o...,153,Male,1
2,HOPPER,3,5,"Hey, dipshits! The men drop their tools and st...",hey dipshit the man drop -pron- tool and stand...,72,Male,1
3,HOPPER,3,5,"Dr. Alexei steps forward, putting himself betw...",alexei step forward put -pron- between hopper ...,148,Male,1
4,HOPPER,3,5,English! You speak English? Alexei tries to ca...,english -pron- speak english alexei try to cal...,61,Male,1
...,...,...,...,...,...,...,...,...
5446,SUSAN,2,9,See -- pretty -- !!,see pretty,10,Female,1
5447,SUSAN,2,9,... Now -- you surrrre you don't want to wear...,now -pron- surrrre -pron- do want to wear a dr...,103,Female,1
5448,CLAUDIA,2,9,Where did you see it last?,where do -pron- see -pron- last,31,Female,1
5449,CLAUDIA,2,9,What's in there that's so important anyway? ...,what in there that so important anyway -pron- ...,123,Female,1


In [106]:
#pip install textstat

     |████████████████████████████████| 105 kB 43.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 47.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [107]:
import textstat
df_sh2['grade'] = df_sh2.Dialog.apply(lambda x: textstat.flesch_kincaid_grade(x))
df_sh2['difficulty'] = df_sh2.Dialog.apply(lambda x: textstat.difficult_words(x))

In [108]:
df_sh2

,Character_Fix,Season,Episode,Dialog,token_lemma,length,Gender,Age,grade,difficulty
0,HOPPER,3,5,Looks like somebody’s home. As the truck parks...,look like somebody home as the truck park -pro...,322,Male,1,2.6,11
1,HOPPER,3,5,Where’s that coming from? Joyce pauses at the ...,where that come from joyce pause at the foot o...,153,Male,1,2.8,3
2,HOPPER,3,5,"Hey, dipshits! The men drop their tools and st...",hey dipshit the man drop -pron- tool and stand...,72,Male,1,0.3,1
3,HOPPER,3,5,"Dr. Alexei steps forward, putting himself betw...",alexei step forward put -pron- between hopper ...,148,Male,1,4.3,6
4,HOPPER,3,5,English! You speak English? Alexei tries to ca...,english -pron- speak english alexei try to cal...,61,Male,1,5.2,2
...,...,...,...,...,...,...,...,...,...,...
5446,SUSAN,2,9,See -- pretty -- !!,see pretty,10,Female,1,2.9,0
5447,SUSAN,2,9,... Now -- you surrrre you don't want to wear...,now -pron- surrrre -pron- do want to wear a dr...,103,Female,1,0.1,2
5448,CLAUDIA,2,9,Where did you see it last?,where do -pron- see -pron- last,31,Female,1,-1.5,0
5449,CLAUDIA,2,9,What's in there that's so important anyway? ...,what in there that so important anyway -pron- ...,123,Female,1,3.2,3


In [109]:
#adding the compound score on again but with the token lemma

df_sh2['polarity'] = df_sh2['token_lemma'].apply(polarity)  #polarity checking
df_sh2['compound']  = df_sh2['polarity'].apply(lambda score_dict: score_dict['compound'])

df_sh2['sentiment'] = df_sh2['compound'].apply(lambda x: "Positive" if x>=0.5 \
                                                     else("Negative" if x<=-0.5 \
                                                     else "Neutral"))
print("polarity checked successfull")

polarity checked successfull


In [110]:
#removing short spoken lines like, no, what, so , hey, yep etc 
#basically when someone spoke 3 words or less
df_sh2['length_spk'] = df_sh2['token_lemma'].apply(lambda x: len(x.split()))

In [111]:
df_sh2 = df_sh2[df_sh2['length_spk'] >3].copy()

In [112]:
df_sh2.head()

,Character_Fix,Season,Episode,Dialog,token_lemma,length,Gender,Age,grade,difficulty,polarity,compound,sentiment,length_spk
0,HOPPER,3,5,Looks like somebody’s home. As the truck parks...,look like somebody home as the truck park -pro...,322,Male,1,2.6,11,"{'neg': 0.089, 'neu': 0.881, 'pos': 0.03, 'com...",-0.3612,Neutral,59
1,HOPPER,3,5,Where’s that coming from? Joyce pauses at the ...,where that come from joyce pause at the foot o...,153,Male,1,2.8,3,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,31
2,HOPPER,3,5,"Hey, dipshits! The men drop their tools and st...",hey dipshit the man drop -pron- tool and stand...,72,Male,1,0.3,1,"{'neg': 0.255, 'neu': 0.588, 'pos': 0.157, 'co...",-0.2500,Neutral,15
3,HOPPER,3,5,"Dr. Alexei steps forward, putting himself betw...",alexei step forward put -pron- between hopper ...,148,Male,1,4.3,6,"{'neg': 0.17, 'neu': 0.83, 'pos': 0.0, 'compou...",-0.5719,Negative,25
4,HOPPER,3,5,English! You speak English? Alexei tries to ca...,english -pron- speak english alexei try to cal...,61,Male,1,5.2,2,"{'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'comp...",0.3182,Neutral,10


In [113]:
check_appearance = df_sh2.groupby(['Character_Fix'])['Character_Fix'].count()

In [114]:
appearance = pd.DataFrame({'Character_Fix':check_appearance.index, 'Num':check_appearance.values})

In [115]:
appearance.sort_values(by='Num')

,Character_Fix,Num
3,ANDY,1
52,TROY,1
51,TOMMY,1
29,JAMES,1
13,DEMOGORGON,1
55,VICTOR CREEL,2
54,VICKIE,2
49,TERRY,2
8,BOB,2
25,HENRY,2


In [116]:
df_sh2 = df_sh2.merge(appearance, how="left", on ='Character_Fix').copy()

In [117]:
df_sh2 = df_sh2[df_sh2['Num']> 2].copy()

In [118]:
df_sh2['Gender'].replace(['Female ', 'Female','Male','Male ','Neutral'],
                        [1, 1,2,2,0], inplace=True)

In [119]:
df_sh2['sentiment'].replace(['Negative', 'Neutral','Positive'],
                        [-1,0,1], inplace=True)

In [120]:
df_sh2.drop(columns=['Dialog','length','polarity'],inplace=True, axis = 1)

In [121]:
df_sh2.head()

,Character_Fix,Season,Episode,token_lemma,Gender,Age,grade,difficulty,compound,sentiment,length_spk,Num
0,HOPPER,3,5,look like somebody home as the truck park -pro...,2,1,2.6,11,-0.3612,0,59,501
1,HOPPER,3,5,where that come from joyce pause at the foot o...,2,1,2.8,3,0.0000,0,31,501
2,HOPPER,3,5,hey dipshit the man drop -pron- tool and stand...,2,1,0.3,1,-0.2500,0,15,501
3,HOPPER,3,5,alexei step forward put -pron- between hopper ...,2,1,4.3,6,-0.5719,-1,25,501
4,HOPPER,3,5,english -pron- speak english alexei try to cal...,2,1,5.2,2,0.3182,0,10,501


### Let's vectorize and train with our hyper parameters designated above before we re-tune with this new df with more features and removal of characters who appeared two times or less and removal of character rows when they spoke who spoke 3 words or less.

In [ ]:
#df_sh2.shape

In [ ]:
#df_train2 = df_sh2.drop(columns=['Character_Fix'],axis=1)

In [122]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(x_var_2d, y_var, test_size = .2)
train_df2_fin, test_df2_fin = \
              np.split(df_sh2.sample(frac=1, random_state=RANDOM_SEED), 
                       [int(.8*len(df_sh2))])


In [123]:
print(len(train_df2_fin), len(test_df2_fin))

3712 928


In [124]:
vectorizer2 = TfidfVectorizer(min_df = 20, stop_words = 'english')
X_train2 = vectorizer2.fit_transform(train_df2_fin.token_lemma)
X_test2 = vectorizer2.transform(test_df2_fin.token_lemma)

In [125]:
y_train2=train_df2_fin['Character_Fix'].values
y_test2=test_df2_fin['Character_Fix'].values

In [ ]:
# X_scaler2 = preprocessing.StandardScaler(with_mean=False).fit_transform(X_train2)
# X_test_Scaler2 = preprocessing.StandardScaler(with_mean=False).fit_transform(X_test2)

In [127]:
df_tf2_fin = pd.DataFrame(X_train2.toarray(), columns=vectorizer2.get_feature_names_out())
train_df2_fin.reset_index(inplace=True)
fin_train_w_feat = pd.concat([df_tf2_fin,train_df2_fin],axis=1)
fin_train_w_feat.drop(['token_lemma','index'], axis=1, inplace=True)

#doing the same to the test data
df_tf_test_fin = pd.DataFrame(X_test2.toarray(), columns=vectorizer2.get_feature_names_out())
final_df_test = test_df2_fin.reset_index()
test_w_feat_final = pd.concat([df_tf_test_fin,final_df_test],axis=1)
test_w_feat_final.drop(['token_lemma','index'], axis=1, inplace=True)
#drpping character label names
test_w_feat_final.drop(['Character_Fix'],axis=1,inplace = True)
fin_train_w_feat.drop(['Character_Fix'], axis=1, inplace=True)

In [128]:
#now we try it out



RCF_model_tuned_v2 = RandomForestClassifier(n_estimators=100, \
                              random_state=RANDOM_SEED,\
                              max_features='sqrt').fit(fin_train_w_feat, y_train2)

y_pred_RCF_model_tuned2 = RCF_model_tuned_v2.predict(test_w_feat_final)


accm_tuned2 = metrics.accuracy_score(y_test2,y_pred_RCF_model_tuned2)

print(accm_tuned2)

acc_train2 = RCF_model_tuned_v2.score(fin_train_w_feat, y_train2)

print(acc_train2)














# lg_model_tuned_v2 = LogisticRegression(max_iter = 500,
#                               random_state=RANDOM_SEED,
#                               solver = 'liblinear',penalty= 'l1',
#                               C = 0.1).fit(fin_train_w_feat, y_train2)

# y_pred_lg_model_tuned2 = lg_model_tuned_v2.predict(test_w_feat_final)

# accm_tuned2 = metrics.accuracy_score(y_test2,y_pred_lg_model_tuned2)

# print(accm_tuned2)

# acc_train2 = lg_model_tuned_v2.score(fin_train_w_feat, y_train2)

# print(acc_train2)

0.8868534482758621
1.0


done

#### Tuning with our model which has features and row removed (based on too few appearences overall and too few words spoken

In [ ]:
# X_train_scaler_final = preprocessing.StandardScaler(with_mean=False).fit_transform(fin_train_w_feat)
# X_test_scaler_final = preprocessing.StandardScaler(with_mean=False).fit_transform(test_w_feat_final)

param_grid_ 2 = {'n_estimators': [500, 800, 1500, 2500, 5000],
               'max_features': ['auto', 'sqrt', 'log2'],
               'max_depth':[10, 20, 30, 40, 50],
               'min_samples_split': [2, 5, 10, 15, 20],
               'min_samples_leaf': [1, 2, 5, 10, 15]
              }


RCF2 = RandomForestClassifier(random_state=RANDOM_SEED)
RCF_random2 = RandomizedSearchCV(estimator=RCF2,
                                param_distributions=param_grid_,
                               n_iter=100,
                               cv=5,
                               verbose=2,
                               random_state=RANDOM_SEED,
                               )

RCF_random2.fit(fin_train_w_feat,y_train2)

print(RCF_random2.best_score_)




In [143]:
#now we try it out



RCF_model_tuned3 = RandomForestClassifier(n_estimators=100, \
                              random_state=RANDOM_SEED,\
                              max_features='sqrt').fit(fin_train_w_feat, y_train2)

y_pred_RCF_model_tuned3 = RCF_model_tuned3.predict(test_w_feat_final)


accm_tuned3 = metrics.accuracy_score(y_test2,y_pred_RCF_model_tuned3)

print(accm_tuned3)

acc_train3 = RCF_model_tuned3.score(fin_train_w_feat, y_train2)

print(acc_train3)




0.8868534482758621


ValueError: X has 275 features, but DecisionTreeClassifier is expecting 290 features as input.

In [144]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
print('Classification report of imbalanced logistic regression \n',classification_report(y_test2,
                                                                                         y_pred_lg_model_tuned3,
                                                                                        zero_division = 0))

cm = confusion_matrix(y_test2,y_pred_RCF_model_tuned3,normalize='true',
                      labels=RCF_model_tuned_v3.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=RCF_model_tuned_v3.classes_)
fig2, ax2 = plot.subplots(figsize=(12,12))
disp.plot(cmap='magma', ax=ax2,xticks_rotation='vertical')
plt.grid(False)


NameError: name 'y_pred_lg_model_tuned3' is not defined

In [ ]:
#from sklearn.metrics import RocCurveDisplay
#RocCurveDisplay.from_predictions(y_test2,y_pred_lg_model_tuned3)

#plt.show()

### Liblinear had the same test score but slightly lower training we will keep with Saga

### Last filter to help the classification with features added to the data is to only use top 20 Characters, this value is based off length of lines spoken summed for a character across the entire data set.  Hopefully the classification will be helped by this filter on the data since it will have more lines to learn off of for a character. Where it is really failing on characters with not a lot of line or characters who have spoken to the same character often, example is Eleven and Karen Wheeler who speak to Mike often and in a similar manner.  While Karen is Mike's mother and Eleven is Mike's girlfriend - although Karen has less lines than Eleven both are still main characters and speak to Mike in a similar manner - for some reason this is not surprising this is happening.  

### It might be a nice add on to explore this failure by taking the characters and seeing their cosine similarity scores based on lines spoken compared to other characters for the Top 20.

In [130]:
train_top20_fin=train_df2_fin.copy()

train_top20_fin['TOP_20'] = train_top20_fin['Character_Fix'].apply(lambda x: \
                                True if x in top20chars else False)

print(len(top20chars))

train_top20_fin = train_top20_fin[train_top20_fin['TOP_20']==True].copy()

print(len(list(train_top20_fin.Character_Fix.unique())))

train_top20_fin.head()

20
20


,index,Character_Fix,Season,Episode,token_lemma,Gender,Age,grade,difficulty,compound,sentiment,length_spk,Num,TOP_20
0,2858,MIKE,1,2,-pron- just -pron- do feel good -pron- wake up...,2,2,6.8,1,-0.6469,-1,48,442,True
2,2681,NANCY,3,1,no -pron- mean -pron- can be late -pron- like ...,1,2,0.3,0,-0.2023,0,15,309,True
3,3106,KAREN,1,2,-pron- not mad at -pron-,1,1,-1.9,0,0.3875,0,5,87,True
4,774,BILLY,3,8,go the mind flayer tentacle have nearly envelo...,2,2,5.6,3,0.0000,0,12,116,True
5,474,HOPPER,3,8,murray all set down there,2,1,0.5,1,0.0000,0,5,501,True


In [131]:
train_top20_fin.groupby(['Character_Fix'])['length_spk'].min()

Character_Fix
BILLY          4
DR BRENNER     4
DR OWENS       4
DUSTIN         4
ELEVEN         4
ERICA          4
HOPPER         4
JONATHAN       4
JOYCE          4
KAREN          4
LUCAS          4
MAX            4
MAYOR KLINE    4
MIKE           4
MURRAY         4
NANCY          4
ROBIN          4
STEVE          4
TOM            7
WILL           4
Name: length_spk, dtype: int64

In [132]:
test_top20_fin=test_df2_fin.copy()

test_top20_fin['TOP_20'] = test_top20_fin['Character_Fix'].apply(lambda x: \
                                True if x in top20chars else False)

#print(len(top20chars))

test_top20_fin = test_top20_fin[test_top20_fin['TOP_20']==True].copy()

print(len(list(test_top20_fin.Character_Fix.unique())))

test_top20_fin.head()

20


,Character_Fix,Season,Episode,token_lemma,Gender,Age,grade,difficulty,compound,sentiment,length_spk,Num,TOP_20
3076,NANCY,1,2,nice the search party have move to the immedia...,1,2,6.6,12,0.8720,1,45,309,True
4260,LUCAS,4,8,-pron- identify -pron- most likely next victim...,2,2,6.4,3,-0.3321,0,8,216,True
4296,JONATHAN,4,8,sure -pron- get -pron- measurement right,2,2,2.1,1,0.3182,0,6,185,True
2279,WILL,3,7,-pron- build -pron- to stop eleven,2,2,0.9,0,-0.2960,0,6,151,True
1199,TOM,3,4,no as heather stand up billy kneel in front of...,2,1,0.5,1,-0.5574,-1,16,30,True


In [134]:
vectorizer_final_top20 = TfidfVectorizer(min_df = 20, stop_words = 'english')
X_train_top20 = vectorizer_final_top20.fit_transform(train_top20_fin.token_lemma)
X_test_top20 = vectorizer_final_top20.transform(test_top20_fin.token_lemma)

y_train_top20=train_top20_fin['Character_Fix'].values
y_test_top20=test_top20_fin['Character_Fix'].values

#scaling since it is sparce and the tuner didnt like it not scaled at any variatio or 
#iteration before
X_scaler_top20 = preprocessing.StandardScaler(with_mean=False).fit_transform(X_train_top20)
X_test_Scaler_top20 = preprocessing.StandardScaler(with_mean=False).fit_transform(X_test_top20)

#adding features back on to both train and test
df_top20_fin = pd.DataFrame(X_scaler_top20.toarray(), \
                          columns=vectorizer_final_top20.get_feature_names())
train_top20_fin.reset_index(inplace=True)
X_train_20fin = pd.concat([df_top20_fin,train_top20_fin],axis=1)
X_train_20fin.drop(['token_lemma','index','TOP_20'], axis=1, inplace=True)

#doing the same to the test data-adding features back on
df_tf_test_fin = pd.DataFrame(X_test_Scaler_top20.toarray(), columns=vectorizer_final_top20.get_feature_names())
test_top20_fin.reset_index(inplace = True)
X_test_20fin = pd.concat([df_tf_test_fin,test_top20_fin],axis=1)
X_test_20fin.drop(['token_lemma','index','TOP_20'], axis=1, inplace=True)
#drpping character label names
X_test_20fin.drop(['Character_Fix'],axis=1,inplace = True)
X_train_20fin.drop(['Character_Fix'], axis=1, inplace=True)

In [135]:
X_train_20fin.isna().values.any()

False

In [136]:
X_test_20fin.isna().values.any()

False

### comment drop level_0 out if not needed be for shape error

In [137]:
#X_train_20fin

In [138]:
X_train_20fin.drop(['level_0'], axis=1, inplace=True)

In [142]:
RCF_final_top20 = RandomForestClassifier(n_estimators=100, \
                              random_state=RANDOM_SEED,\
                              max_features='sqrt').fit(X_train_20fin, y_train_top20)

y_pred_rcf_top20 = RCF_final_top20.predict(X_test_20fin)


accm20 = metrics.accuracy_score(y_test_top20,y_pred_rcf_top20)


RCF_model_SCR20 = RCF_final_top20.score(X_test_20fin, y_test_top20)



RCF_model_train_scr_top20 = RCF_final_top20.score(X_train_20fin, y_train_top20)


print("Train Accuracy Score:",RCF_model_train_scr_top20)

print("Test Accuracy Score:",accm20)

print("Test Accuracy Score:",RCF_model_SCR20)

Train Accuracy Score: 1.0
Test Accuracy Score: 0.9142185663924794
Test Accuracy Score: 0.9142185663924794


DONE

print('Classification report of imbalanced logistic regression \n',classification_report(y_test2,
                                                                                         y_pred_lg_model_tuned3,
                                                                                        zero_division = 0))

cm20 = confusion_matrix(y_test_top20,y_pred_lg_top20,normalize='true',
                      labels=lg_final_top20.classes_)
disp20 = ConfusionMatrixDisplay(confusion_matrix=cm20,
                              display_labels=lg_final_top20.classes_)
fig2, ax2 = plot.subplots(figsize=(14,14))
disp20.plot(cmap='magma', ax=ax2,xticks_rotation='vertical')
plt.grid(False)

In [ ]:
param_grid_top20 = {'C': [0.01,0.1,0.5,1.0,5,10],
                'penalty':['l1','l2'],
                'solver':['liblinear','lbfgs','sag','saga'],
                }
#bow_search = model_selection.GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)
final_tune_top20 = model_selection.GridSearchCV(LogisticRegression(random_state = RANDOM_SEED,\
                                                              max_iter = 250),n_jobs = 50,
                                                              param_grid=param_grid_top20,
                                                              scoring = 'accuracy',
                                                              cv = 2)

final_tune_top20.fit(X_train_20fin, y_train_top20)

print("Tuned Hyperparameters:", final_tune_top20.best_params_)
print("Accuracy:",final_tune_top20.best_score_)

### So with the chopped down data set to only top 20 characters it is saying the accuracy could be 70% which is better but still doesnt make complete sense.  It is also saying the new solver should be liblinear.  We also need to check with class_weight =balanced and then the max_iter but we will do that when we figure out which data/features are best for our model. Also C is now 1.0 instead of 0.1 or 0.5.

In [ ]:
lg_final_top20 = LogisticRegression(max_iter = 1000,
                              random_state=RANDOM_SEED,
                              solver = 'liblinear',penalty= 'l1',
                              C = 1.0).fit(X_train_20fin, y_train_top20)

y_pred_lg_top20 = lg_final_top20.predict(X_test_20fin)

accm20 = metrics.accuracy_score(y_test_top20,y_pred_lg_top20)
lg_model_scr20 = lg_final_top20.score(X_test_20fin, y_test_top20)

lg_model_Train_scr_top20 = lg_final_top20.score(X_train_20fin, y_train_top20)

print("Train Accuracy Score:",lg_model_Train_scr_top20)

print("Test Accuracy Score:",accm20)

print("Test Accuracy Score:",lg_model_scr20)

In [ ]:
print('Classification report of imbalanced logistic regression \n',classification_report(y_test2,
                                                                                         y_pred_lg_model_tuned3,
                                                                                        zero_division = 0))

cm20v2 = confusion_matrix(y_test_top20,y_pred_lg_top20,normalize='true',
                      labels=lg_final_top20.classes_)
disp20v2 = ConfusionMatrixDisplay(confusion_matrix=cm20,
                              display_labels=lg_final_top20.classes_)
fig2, ax2 = plot.subplots(figsize=(14,14))
disp20v2.plot(cmap='magma', ax=ax2,xticks_rotation='vertical')
plt.grid(False)
plt.rcParams.update({'text.color': "white",
                     'axes.labelcolor': "white"})
plt.title("Logistic Regression: Top 20 Characters only and All Features\nC: 1.0, solver:liblinear, penalty: L1, max_iter = 1000",
          {'fontsize': 12,'color': "white"})


### Wow the test accuracy went up to 75.44% with using just rows/data for the top 20 characters in the show and the training score shot up to 93.1%.  Regardless of changes we make such a feature additions such as gender, age (age group), season, episode, text difficulty, text grade level, sentiment compound value, sentiment category based on compound value, length of token for a line, total num words spoken and removal of instances where the character spoke 3 words or less based on token_lemma length.

### Lets look at the correlation of the different features and categorize the character names

In [ ]:
from dython.nominal import associations
from dython.nominal import identify_nominal_columns

train_corre=train_df2_fin.copy()

train_corre['TOP_20'] = train_corre['Character_Fix'].apply(lambda x: \
                                True if x in top20chars else False)
test_corre=test_df2_fin.copy()

test_corre['TOP_20'] = test_corre['Character_Fix'].apply(lambda x: \
                                True if x in top20chars else False)

check_corre = pd.concat([train_corre, test_corre], ignore_index=True, axis=0)

check_corre.drop(columns=['index','token_lemma'],axis = 1,inplace = True)

f = lambda x: 1 if x==True else 0
check_corre['TOP_20'] = check_corre['TOP_20'].apply(f)
print(check_corre.shape)
#!pip3 install dython

categorical_features=identify_nominal_columns(check_corre)
print(categorical_features)

associations(check_corre, nominal_columns='auto', 
             numerical_columns=None, mark_columns=False, 
             nom_nom_assoc='cramer', num_num_assoc='pearson', 
             cramers_v_bias_correction=True, nan_strategy=False,
             nan_replace_value=False, ax =sns.set(rc ={'xtick.labelcolor':'white',
                                                        'ytick.labelcolor':'white',
                                                       'figure.facecolor':'black'}),
             figsize=(12,10), annot=True, fmt='.2f', cmap="inferno",
             sv_color='black', cbar=True, vmax=1.0, vmin=None, plot=True,
             compute_only=False, clustering=False, title=None, filename=None)

### As we can see the Character Fix column does has correlation between the various features we added especially the season, episode, gender, age and num of words spoken in total along with top 20.  But we only kept where top 20 was True lets see what that correlation looks like on that dataframe instead, maybe we need to not filter out the data but run the model with the column as a feature in total.   

In [ ]:
train_20_corre = train_top20_fin.copy()
test_20_corre = test_top20_fin.copy()
train_20_corre['TOP_20'] = train_20_corre['Character_Fix'].apply(lambda x: \
                                True if x in top20chars else False)

test_20_corre['TOP_20'] = test_20_corre['Character_Fix'].apply(lambda x: \
                                True if x in top20chars else False)

check_corre20 = pd.concat([train_20_corre, test_20_corre], ignore_index=True, axis=0)

check_corre20.drop(columns=['level_0','index','token_lemma'],axis = 1,inplace = True)
print(check_corre20.shape)
f = lambda x: 1 if x==True else 0
check_corre20['TOP_20'] = check_corre20['TOP_20'].apply(f)

#!pip3 install dython

categorical_features20=identify_nominal_columns(check_corre20)
print(categorical_features20)

associations(check_corre20, nominal_columns='auto', 
             numerical_columns=None, mark_columns=False, 
             nom_nom_assoc='cramer', num_num_assoc='pearson', 
             cramers_v_bias_correction=True, nan_strategy=False,
             nan_replace_value=False, ax =sns.set(rc ={'xtick.labelcolor':'white',
                                                        'ytick.labelcolor':'white',
                                                       'figure.facecolor':'black'}),
             figsize=(12,10), annot=True, fmt='.2f', cmap="inferno",
             sv_color='black', cbar=True, vmax=1.0, vmin=None, plot=True,
             compute_only=False, clustering=False, title=None, filename=None)

### Some of the correlations are the same but some are lower for Character fix when just using top20 character data only, such as the drop in correlation for season and episode and the gender correlation to num of total lines spoken for a character was almost neutral a -0.06 but now its weighted at 0.28.  I think filter for just top 20 was not correct and we shoul have just left it as a feature instead.

### Running the model again but with top 20 as feature instead of a filter

In [ ]:
#train_corre

In [ ]:
#train_corre
#test_corre
vectorizer_final_model = TfidfVectorizer(min_df = 20, stop_words = 'english')
X_train_Final = vectorizer_final_model.fit_transform(train_corre.token_lemma)
X_test_Final = vectorizer_final_model.transform(test_corre.token_lemma)

y_train_Final=train_corre['Character_Fix'].values
y_test_Final=test_corre['Character_Fix'].values

#scaling since it is sparce and the tuner didnt like it not scaled at any variatio or 
#iteration before
X_scaler_Final = preprocessing.StandardScaler(with_mean=False).fit_transform(X_train_Final)
X_test_scaler_Final = preprocessing.StandardScaler(with_mean=False).fit_transform(X_test_Final)

#adding features back on to both train and test
df_Final = pd.DataFrame(X_scaler_Final.toarray(), \
                          columns=vectorizer_final_model.get_feature_names_out())
train_corre.reset_index(inplace=True)
X_train_Final_Model = pd.concat([df_Final,train_corre],axis=1)
X_train_Final_Model.drop(['token_lemma','index','TOP_20'], axis=1, inplace=True)

#doing the same to the test data-adding features back on
df_test_Final = pd.DataFrame(X_test_scaler_Final.toarray(), columns=vectorizer_final_model.get_feature_names_out())
test_corre.reset_index(inplace = True)
X_test_Final_Model = pd.concat([df_test_Final,test_corre],axis=1)
X_test_Final_Model.drop(['token_lemma','index','TOP_20'], axis=1, inplace=True)
#drpping character label names
X_test_Final_Model.drop(['Character_Fix'],axis=1,inplace = True)
X_train_Final_Model.drop(['Character_Fix'], axis=1, inplace=True)

In [ ]:
X_train_Final_Model.drop(['level_0'], axis=1, inplace=True)

In [ ]:
#X_test_Final_Model.drop(['level_0'],axis=1,inplace = True)

In [ ]:
lg_final_features = LogisticRegression(max_iter = 500,
                              random_state=RANDOM_SEED,
                              solver = 'liblinear',penalty= 'l1',
                              C = 1.0).fit(X_train_Final_Model, y_train_Final)

y_pred_FINALft = lg_final_features.predict(X_test_Final_Model)

accmfin = metrics.accuracy_score(y_test_Final,y_pred_FINALft)
lg_model_scrfin = lg_final_features.score(X_test_Final_Model, y_test_Final)

lg_model_Train_scr_Final = lg_final_features.score(X_train_Final_Model, y_train_Final)

print("Train Accuracy Score:",lg_model_Train_scr_Final)

print("Test Accuracy Score:",accmfin)

print("Test Accuracy Score:",lg_model_scrfin)

### Not bad for keeping all the characters, all the featues and using top 20 as a feature instead of a filter but I think we will keep the top 20 as a filter with all the features since overall it performed the best so far at 76% on test data.

### Failure Analysis or Reasoning for mismarked lines spoken by a charcter

### checking cosine similarity to see why certain characters lines are being picked up by a different characters

In [ ]:
sentiment_df = sent_dict[['Character_Fix','Dialog_clean','compound']].copy()
vectorizer_sentiment = TfidfVectorizer(stop_words = 'english')
sent_vect = vectorizer_sentiment.fit_transform(sentiment_df.Dialog_clean)


#adding features back on to both train and test
df_s = pd.DataFrame(sent_vect.toarray(), \
                          columns=vectorizer_sentiment.get_feature_names_out())
#train_corre.reset_index(inplace=True)
sentiment_send = pd.concat([sentiment_df,df_s],axis=1)
sentiment_send.drop(['Dialog_clean'], axis=1, inplace=True)
sentiment_send_fin = pd.merge(left = sentiment_send,right =gender, on=['Character_Fix'])
sentiment_send_fin['Age'] = sentiment_send_fin['Age'].apply(lambda x: 1 if x=="Female" \
                                                     else(2 if x=="Negative" \
                                                     else 0))

In [ ]:
#sentiment_send.set_index('Character_Fix')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#from scipy.spatial.distance import cosine

def calc_cos_sim(df):
    """
    Takes in a DataFrame and computes all pairwise Cosine Similarity
    """
    s = pd.Series(['BILLY', 'DUSTIN', 'ELEVEN', 'ERICA', 'HOPPER', 'JONATHAN',
       'JOYCE', 'KAREN', 'LUCAS', 'MAX', 'MAYOR KLINE', 'MIKE', 'MURRAY',
       'NANCY', 'ROBIN', 'STEVE', 'TOM', 'WILL'])
    cols = df.columns.to_list()
    cos_sim_df = pd.DataFrame(cosine_similarity(df),columns=s)
    cos_sim_df.set_index([s])
    
    return cos_sim_df.set_index([s]).fillna(1.0)

fail_analysis_reason = calc_cos_sim(sentiment_send.set_index('Character_Fix'))

### make final dataframe of different score iterations for comparison in report and to other classification models we use/test

In [ ]:
fail_analysis_reason.to_csv("sentiment_sim_fail_label.csv")